In [1]:
!pip install -q mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

In [4]:
import os
import cv2
import pickle
import time
import warnings
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from google.colab.patches import cv2_imshow

In [5]:
# exmaple_img = cv2.imread('3')
# cv2_imshow(exmaple_img)

In [6]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

def get_lip_landmark_mediapipe(img, visualize=True):
    """
    Using MediaPipe to extract lip landmark features.
    Number of points: 40
    :param img:
    :return:
    """
    # Convert the color space from BGR to RGB
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
    detection_result = detector.detect(image)

    if visualize:
        # Prepare to collect landmark data
        visualize_lip_points = []

    lip_points = []

    lip_connections = [
        (61, 146), (146, 91), (91, 181), (181, 84), (84, 17),
        (17, 314), (314, 405), (405, 321), (321, 375), (375, 291),
        (61, 185), (185, 40), (40, 39), (39, 37), (37, 0),
        (0, 267), (267, 269), (269, 270), (270, 409), (409, 291),
        (78, 95), (95, 88), (88, 178), (178, 87), (87, 14),
        (14, 317), (317, 402), (402, 318), (318, 324), (324, 308),
        (78, 191), (191, 80), (80, 81), (81, 82), (82, 13),
        (13, 312), (312, 311), (311, 310), (310, 415), (415, 308)
    ]
    unique_indices = set([idx for pair in lip_connections for idx in pair])  # Flatten and remove duplicates

    if len(detection_result.face_landmarks) != 0:
        for i, landmark in enumerate(detection_result.face_landmarks[0]):
            if i in unique_indices:
                lip_points.append((landmark.x, landmark.y))

                if visualize:
                    # Draw lip landmarks on the lip
                    visualize_x = landmark.x * img.shape[1]
                    visualize_y = landmark.y * img.shape[0]
                    cv2.circle(img, (int(visualize_x), int(visualize_y)), 2, (0, 255, 0), -1)

        # Min-max scale lip points
        min_x = min(lip_points, key=lambda p: p[0])[0]
        max_x = max(lip_points, key=lambda p: p[0])[0]
        min_y = min(lip_points, key=lambda p: p[1])[1]
        max_y = max(lip_points, key=lambda p: p[1])[1]
        scaled_lip_points = [
            (
                (x - min_x) / (max_x - min_x) if max_x != min_x else 0,
                (y - min_y) / (max_y - min_y) if max_y != min_y else 0,
            )
            for (x, y) in lip_points
        ]

        # Draw scaled lip points
        scale = 100
        for i, (scaled_x, scaled_y) in enumerate(scaled_lip_points):
            cv2.circle(img, (int(scaled_x * scale), int(scaled_y * scale)), 3, (255, 255, 0), -1)

        return lip_points, scaled_lip_points, img
    else:
        return None, None, None

In [7]:
# example_img = cv2.imread('3')
# new_lip_points, new_scaled_lip_points, new_after_img = get_lip_landmark_mediapipe_new(example_img)
# cv2_imshow(new_after_img)

In [10]:
def get_lip_points_video(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    lip_points_video = []
    img_list = []
    missing_frame = False
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save individual sampled frames to the specified directory
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, lip_points_frame, img = get_lip_landmark_mediapipe(img, visualize=True)
        # _, cropped_img = detect_face(img)
        # lip_points_frame, img = get_lip_landmark_mediapipe(cropped_img, visualize=True)
        # time.sleep(1)
        if lip_points_frame is None:
            missing_frame = True
            break
        else:
            img_list.append(img)
            lip_points_video.extend(lip_points_frame)
    # Release the video capture object
    cap.release()
    if not missing_frame:
        return lip_points_video, img_list
    else:
        return None, None

In [11]:
video_dir_path = '/content/drive/MyDrive/lipread_mp4'
label_names = [label for label in os.listdir(video_dir_path) if label != '.DS_Store']
train_pickle_output_dir = '/content/drive/MyDrive/lipreader_pickle/train'
val_pickle_output_dir = '/content/drive/MyDrive/lipreader_pickle/val'
test_pickle_output_dir = '/content/drive/MyDrive/lipreader_pickle/test'
train_frame_output_dir = '/content/drive/MyDrive/lipreader_frame/train'
val_frame_output_dir = '/content/drive/MyDrive/lipreader_frame/val'
test_frame_output_dir = '/content/drive/MyDrive/lipreader_frame/test'

LABEL_LIMIT = ['ANSWER', 'SOMETHING', 'WEEKEND', 'WATER', 'OFFICE', 'THOSE', 'TODAY', 'ENERGY', 'TOMORROW', 'NEVER']

for label in label_names:
    print(label)
    source_directory = os.path.join(video_dir_path, label)

    # train
    if label not in LABEL_LIMIT:
        continue
    train_directory = os.path.join(source_directory, 'train/')
    label_folder = os.path.join(train_pickle_output_dir, label.split('.')[0])
    if os.path.exists(label_folder):
        extracted_files = [pkl_file.split('.')[0] for pkl_file in os.listdir(label_folder) if label != '.DS_Store']
    else:
        extracted_files = []
    video_files = [file for file in os.listdir(train_directory) if file.endswith('.mp4')]
    for video_file in video_files:
        if video_file.split('.')[0] not in extracted_files:
            print('train:', video_file)
            video_file_path = os.path.join(train_directory, video_file)
            lip_points, img_list = get_lip_points_video(video_file_path)
            if lip_points is None:
                print('train:', video_file, ':not complete')
                continue
            else:
                # create label folder
                if not os.path.exists(label_folder):
                    os.mkdir(label_folder)
                train_data = (label.split('.')[0], video_file, lip_points)
                pickle_path = os.path.join(label_folder, video_file.split('.')[0]+'.pkl')
                with open(pickle_path, 'wb') as file:
                    pickle.dump(train_data, file)

                # write to frame
                # for i, img in enumerate(img_list):
                #     img_name = video_file.split('.')[0] + '_' + str(i) + '.jpg'
                #     img_path = os.path.join(train_frame_output_dir, img_name)
                #     cv2.imwrite(img_path, img)
            time.sleep(1)


    # valid
    if label not in LABEL_LIMIT:
        continue
    val_directory = os.path.join(source_directory, 'val/')
    label_folder = os.path.join(val_pickle_output_dir, label.split('.')[0])
    if os.path.exists(label_folder):
        extracted_files = [pkl_file.split('.')[0] for pkl_file in os.listdir(label_folder) if label != '.DS_Store']
    else:
        extracted_files = []
    video_files = [file for file in os.listdir(val_directory) if file.endswith('.mp4')]
    for video_file in video_files:
        if video_file.split('.')[0] not in extracted_files:
            print('val:', video_file)
            video_file_path = os.path.join(val_directory, video_file)
            lip_points, img_list = get_lip_points_video(video_file_path)
            if lip_points is None:
                print('val:', video_file, ':not complete')
                continue
            else:
                # create label folder
                if not os.path.exists(label_folder):
                    os.mkdir(label_folder)
                train_data = (label.split('.')[0], video_file, lip_points)
                pickle_path = os.path.join(label_folder, video_file.split('.')[0]+'.pkl')
                with open(pickle_path, 'wb') as file:
                    pickle.dump(train_data, file)
                # for i, img in enumerate(img_list):
                #     img_name = video_file.split('.')[0] + '_' + str(i) + '.jpg'
                #     img_path = os.path.join(val_frame_output_dir, img_name)
                #     cv2.imwrite(img_path, img)
            time.sleep(1)

    # test
    if label not in LABEL_LIMIT:
        continue
    test_directory = os.path.join(source_directory, 'test/')
    label_folder = os.path.join(test_pickle_output_dir, label.split('.')[0])
    if os.path.exists(label_folder):
        extracted_files = [pkl_file.split('.')[0] for pkl_file in os.listdir(label_folder) if label != '.DS_Store']
    else:
        extracted_files = []
    video_files = [file for file in os.listdir(test_directory) if file.endswith('.mp4')]
    for video_file in video_files:
        if video_file.split('.')[0] not in extracted_files:
            print('test:', video_file)
            video_file_path = os.path.join(test_directory, video_file)
            lip_points, img_list = get_lip_points_video(video_file_path)
            if lip_points is None:
                print('test:', video_file, ':not complete')
                continue
            else:
                # create label folder
                if not os.path.exists(label_folder):
                    os.mkdir(label_folder)
                train_data = (label.split('.')[0], video_file, lip_points)
                pickle_path = os.path.join(label_folder, video_file.split('.')[0]+'.pkl')
                with open(pickle_path, 'wb') as file:
                    pickle.dump(train_data, file)
                # for i, img in enumerate(img_list):
                #     img_name = video_file.split('.')[0] + '_' + str(i) + '.jpg'
                #     img_path = os.path.join(test_frame_output_dir, img_name)
                #     cv2.imwrite(img_path, img)
            time.sleep(1)

Streaming output truncated to the last 5000 lines.
train: TOMORROW_00433.mp4
train: TOMORROW_00227.mp4
train: TOMORROW_00249.mp4
train: TOMORROW_00563.mp4
train: TOMORROW_00353.mp4
train: TOMORROW_00374.mp4
train: TOMORROW_00395.mp4
train: TOMORROW_00226.mp4
train: TOMORROW_00248.mp4
train: TOMORROW_00485.mp4
train: TOMORROW_00604.mp4
train: TOMORROW_00311.mp4
train: TOMORROW_00468.mp4
train: TOMORROW_00143.mp4
train: TOMORROW_00517.mp4
train: TOMORROW_00059.mp4
train: TOMORROW_00562.mp4
train: TOMORROW_00413.mp4
train: TOMORROW_00290.mp4
train: TOMORROW_00017.mp4
train: TOMORROW_00547.mp4
train: TOMORROW_00269.mp4
train: TOMORROW_00590.mp4
train: TOMORROW_00101.mp4
train: TOMORROW_00450.mp4
train: TOMORROW_00484.mp4
train: TOMORROW_00532.mp4
train: TOMORROW_00352.mp4
train: TOMORROW_00185.mp4
train: TOMORROW_00332.mp4
train: TOMORROW_00432.mp4
train: TOMORROW_00412.mp4
train: TOMORROW_00080.mp4
train: TOMORROW_00576.mp4
train: TOMORROW_00603.mp4
train: TOMORROW_00058.mp4
train: TOMORR